In [1]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings

In [2]:
warnings.filterwarnings(action = 'ignore')

In [34]:

import csv
import pandas as pd

import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional

In [4]:
import pandas as pd

In [5]:
dev=pd.read_json('dev/EXIST2023_dev.json')

In [6]:
dev_Yes=pd.read_json('dev/EXIST2023_dev.json')
dev_Yes=dev_Yes.transpose()
dev_Yes=dev_Yes.loc[dev_Yes['lang'].apply(lambda x:x=='en')]
dev_Yes=dev_Yes.loc[dev_Yes['labels_task1'].apply(lambda x:x.count('YES')>3)]
dev_Yes['label1']='YES'
dev_Yes

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
400002,400002,en,@BBCWomansHour @LabWomenDec @EverydaySexism Sh...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, NO]","[REPORTED, JUDGEMENTAL, -, REPORTED, REPORTED, -]","[[IDEOLOGICAL-INEQUALITY], [OBJECTIFICATION], ...",DEV_EN,YES
400003,400003,en,#everydaysexism Some man moving my suitcase in...,6,"[Annotator_776, Annotator_777, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, YES, YES]","[-, REPORTED, REPORTED, REPORTED, REPORTED, JU...","[[-], [STEREOTYPING-DOMINANCE], [OBJECTIFICATI...",DEV_EN,YES
400008,400008,en,@ReproRights @AbortionStories Getting Twitter ...,6,"[Annotator_791, Annotator_122, Annotator_396, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, REPORTED, -, DIRECT, JUDGEMENTAL, DIRECT]","[[-], [OBJECTIFICATION, SEXUAL-VIOLENCE], [-],...",DEV_EN,YES
400011,400011,en,@esjayXX @EcuadorianMum @monsalore They so rem...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, NO, YES]","[-, REPORTED, REPORTED, DIRECT, -, JUDGEMENTAL]","[[-], [OBJECTIFICATION], [MISOGYNY-NON-SEXUAL-...",DEV_EN,YES
400013,400013,en,One of the depressing things about #NotAllMen ...,6,"[Annotator_764, Annotator_765, Annotator_766, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, DIRECT, -, DIRECT, JUDGEMENTAL, JUDGEMENTAL]","[[-], [STEREOTYPING-DOMINANCE], [-], [IDEOLOGI...",DEV_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
400485,400485,en,@YesReallyAngel “Don’t wear a black bra with a...,6,"[Annotator_801, Annotator_182, Annotator_802, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, JUDGEMENTAL, DIRECT, DIRECT, ...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, SEX...",DEV_EN,YES
400486,400486,en,""" get changed , you look like a prostitute . ""...",6,"[Annotator_801, Annotator_182, Annotator_802, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, YES, YES, NO, YES]","[DIRECT, REPORTED, REPORTED, REPORTED, -, DIRECT]","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",DEV_EN,YES
400487,400487,en,made this top and my mom gave me the “you look...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, YES, YES, YES, NO]","[JUDGEMENTAL, REPORTED, REPORTED, REPORTED, RE...","[[OBJECTIFICATION], [OBJECTIFICATION], [OBJECT...",DEV_EN,YES
400488,400488,en,@DawnAnd91320913 I haven't seen anything that ...,6,"[Annotator_776, Annotator_777, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, DIRECT]","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",DEV_EN,YES


In [7]:
dev_No=pd.read_json('dev/EXIST2023_dev.json')
dev_No=dev_No.transpose()
dev_No=dev_No.loc[dev_No['lang'].apply(lambda x:x=='en')]
dev_No=dev_No.loc[dev_No['labels_task1'].apply(lambda x:x.count('NO')>3)]
dev_No['label1']='NO'
dev_No

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
400001,400001,en,"@Mike_Fabricant “You should smile more, love. ...",6,"[Annotator_764, Annotator_765, Annotator_766, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, YES, YES]","[-, -, -, -, REPORTED, DIRECT]","[[-], [-], [-], [-], [IDEOLOGICAL-INEQUALITY, ...",DEV_EN,NO
400004,400004,en,@KolHue @OliverJia1014 lol gamergate the go to...,6,"[Annotator_780, Annotator_781, Annotator_782, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, NO, NO, NO, NO, NO]","[DIRECT, -, -, -, -, -]","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, MIS...",DEV_EN,NO
400005,400005,en,@ShelfStoriesGBL To me this has the same negat...,6,"[Annotator_780, Annotator_781, Annotator_782, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, NO, NO, YES, NO, NO]","[JUDGEMENTAL, -, -, REPORTED, -, -]","[[IDEOLOGICAL-INEQUALITY], [-], [-], [MISOGYNY...",DEV_EN,NO
400006,400006,en,@IrrelevantCmnt @jbo911 @BanButterfly @TheRigh...,6,"[Annotator_780, Annotator_781, Annotator_782, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, NO, NO, NO, NO, NO]","[DIRECT, -, -, -, -, -]","[[STEREOTYPING-DOMINANCE, MISOGYNY-NON-SEXUAL-...",DEV_EN,NO
400007,400007,en,@cathymwafer @andrew_lilico Showing off? The m...,6,"[Annotator_785, Annotator_786, Annotator_787, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",DEV_EN,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
400474,400474,en,#cryptosterone 👈All-new!Twice as many #men inv...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, DIRECT, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY, STERE...",DEV_EN,NO
400476,400476,en,@ScubaMySteve If you ask Cenk as a hypothetica...,6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, NO, NO, NO]","[-, JUDGEMENTAL, -, -, -, -]","[[-], [IDEOLOGICAL-INEQUALITY, MISOGYNY-NON-SE...",DEV_EN,NO
400477,400477,en,@snowflowergomi @minseulsoo @chimsmaze How wil...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, NO, YES]","[-, -, -, -, -, REPORTED]","[[-], [-], [-], [-], [-], [STEREOTYPING-DOMINA...",DEV_EN,NO
400481,400481,en,"Uh, they're allowed to? calling this out makes...",6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, YES, NO, YES, NO]","[-, -, JUDGEMENTAL, -, DIRECT, -]","[[-], [-], [MISOGYNY-NON-SEXUAL-VIOLENCE], [-]...",DEV_EN,NO


In [8]:
devs = [dev_Yes, dev_No]
dev = pd.concat(devs)
dev.reset_index(drop=True)

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
0,400002,en,@BBCWomansHour @LabWomenDec @EverydaySexism Sh...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[YES, YES, NO, YES, YES, NO]","[REPORTED, JUDGEMENTAL, -, REPORTED, REPORTED, -]","[[IDEOLOGICAL-INEQUALITY], [OBJECTIFICATION], ...",DEV_EN,YES
1,400003,en,#everydaysexism Some man moving my suitcase in...,6,"[Annotator_776, Annotator_777, Annotator_195, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, YES, YES]","[-, REPORTED, REPORTED, REPORTED, REPORTED, JU...","[[-], [STEREOTYPING-DOMINANCE], [OBJECTIFICATI...",DEV_EN,YES
2,400008,en,@ReproRights @AbortionStories Getting Twitter ...,6,"[Annotator_791, Annotator_122, Annotator_396, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, REPORTED, -, DIRECT, JUDGEMENTAL, DIRECT]","[[-], [OBJECTIFICATION, SEXUAL-VIOLENCE], [-],...",DEV_EN,YES
3,400011,en,@esjayXX @EcuadorianMum @monsalore They so rem...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, YES, YES, NO, YES]","[-, REPORTED, REPORTED, DIRECT, -, JUDGEMENTAL]","[[-], [OBJECTIFICATION], [MISOGYNY-NON-SEXUAL-...",DEV_EN,YES
4,400013,en,One of the depressing things about #NotAllMen ...,6,"[Annotator_764, Annotator_765, Annotator_766, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, YES, YES, YES]","[-, DIRECT, -, DIRECT, JUDGEMENTAL, JUDGEMENTAL]","[[-], [STEREOTYPING-DOMINANCE], [-], [IDEOLOGI...",DEV_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
439,400474,en,#cryptosterone 👈All-new!Twice as many #men inv...,6,"[Annotator_770, Annotator_771, Annotator_772, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, DIRECT, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY, STERE...",DEV_EN,NO
440,400476,en,@ScubaMySteve If you ask Cenk as a hypothetica...,6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, YES, NO, NO, NO, NO]","[-, JUDGEMENTAL, -, -, -, -]","[[-], [IDEOLOGICAL-INEQUALITY, MISOGYNY-NON-SE...",DEV_EN,NO
441,400477,en,@snowflowergomi @minseulsoo @chimsmaze How wil...,6,"[Annotator_795, Annotator_796, Annotator_797, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, NO, NO, NO, YES]","[-, -, -, -, -, REPORTED]","[[-], [-], [-], [-], [-], [STEREOTYPING-DOMINA...",DEV_EN,NO
442,400481,en,"Uh, they're allowed to? calling this out makes...",6,"[Annotator_805, Annotator_426, Annotator_806, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[NO, NO, YES, NO, YES, NO]","[-, -, JUDGEMENTAL, -, DIRECT, -]","[[-], [-], [MISOGYNY-NON-SEXUAL-VIOLENCE], [-]...",DEV_EN,NO


In [9]:
train_Yes=pd.read_json('training/EXIST2023_training.json')
train_Yes=train_Yes.transpose()
train_Yes=train_Yes.loc[train_Yes['lang'].apply(lambda x:x=='en')]
train_Yes=train_Yes.loc[train_Yes['labels_task1'].apply(lambda x:x.count('YES')>3)]
train_Yes['label1']='YES'
train_Yes

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
200002,200002,en,Writing a uni essay in my local pub with a cof...,6,"[Annotator_391, Annotator_392, Annotator_393, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, NO, YES, YES]","[REPORTED, DIRECT, REPORTED, -, REPORTED, REPO...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, MIS...",TRAIN_EN,YES
200003,200003,en,@UniversalORL it is 2021 not 1921. I dont appr...,6,"[Annotator_397, Annotator_398, Annotator_399, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, NO, YES, NO, YES]","[REPORTED, REPORTED, -, REPORTED, -, JUDGEMENTAL]","[[OBJECTIFICATION, SEXUAL-VIOLENCE], [STEREOTY...",TRAIN_EN,YES
200006,200006,en,According to a customer I have plenty of time ...,6,"[Annotator_409, Annotator_410, Annotator_411, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, REPORTED, REPORTED, REPORTED, REPOR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [S...",TRAIN_EN,YES
200007,200007,en,"So only 'blokes' drink beer? Sorry, but if you...",6,"[Annotator_415, Annotator_416, Annotator_417, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, REPORTED, REPORTED, DIRECT, DIRE...","[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMIN...",TRAIN_EN,YES
200011,200011,en,#EverydaySexism means women usually end up in ...,6,"[Annotator_437, Annotator_438, Annotator_439, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, JUDGEMENTAL, JUDGEMENTAL, JUDGEMENT...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",TRAIN_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
203256,203256,en,idk why y’all bitches think having half your a...,6,"[Annotator_478, Annotator_479, Annotator_480, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, DIRECT, DIRECT, DIRECT, JUDGEMEN...","[[OBJECTIFICATION], [STEREOTYPING-DOMINANCE, S...",TRAIN_EN,YES
203257,203257,en,This has been a part of an experiment with @Wo...,6,"[Annotator_668, Annotator_669, Annotator_670, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, REPORTED, JUDGEMENTAL, DIRECT, J...","[[OBJECTIFICATION], [OBJECTIFICATION], [OBJECT...",TRAIN_EN,YES
203258,203258,en,"""Take me already"" ""Not yet. You gotta be ready...",6,"[Annotator_467, Annotator_468, Annotator_469, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, YES, YES, YES]","[-, DIRECT, -, DIRECT, DIRECT, JUDGEMENTAL]","[[-], [OBJECTIFICATION], [-], [SEXUAL-VIOLENCE...",TRAIN_EN,YES
203259,203259,en,@clintneedcoffee why do you look like a whore?...,6,"[Annotator_674, Annotator_675, Annotator_676, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, DIRECT, JUDGEMENTAL, ...","[[OBJECTIFICATION, SEXUAL-VIOLENCE, MISOGYNY-N...",TRAIN_EN,YES


In [10]:
train_No=pd.read_json('training/EXIST2023_training.json')
train_No=train_No.transpose()
train_No=train_No.loc[train_No['lang'].apply(lambda x:x=='en')]
train_No=train_No.loc[train_No['labels_task1'].apply(lambda x:x.count('NO')>3)]
train_No['label1']='NO'
train_No

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
200008,200008,en,New to the shelves this week - looking forward...,6,"[Annotator_420, Annotator_296, Annotator_421, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, JUDGEMENTAL, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY], [-],...",TRAIN_EN,NO
200010,200010,en,I guess that’s fairly normal for a Neanderthal...,6,"[Annotator_431, Annotator_432, Annotator_433, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, NO, YES, NO]","[-, -, -, -, JUDGEMENTAL, -]","[[-], [-], [-], [-], [OBJECTIFICATION, SEXUAL-...",TRAIN_EN,NO
200023,200023,en,@JavedLSterritt He's really shit on games jour...,6,"[Annotator_496, Annotator_497, Annotator_498, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, JUDGEMENTAL, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY, STERE...",TRAIN_EN,NO
200024,200024,en,"Low key, she was fr created because the Incred...",6,"[Annotator_502, Annotator_503, Annotator_504, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, NO, NO, YES]","[-, -, -, -, -, JUDGEMENTAL]","[[-], [-], [-], [-], [-], [STEREOTYPING-DOMINA...",TRAIN_EN,NO
200025,200025,en,"One of the most important (and of course, unde...",6,"[Annotator_508, Annotator_509, Annotator_510, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",TRAIN_EN,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
203180,203180,en,"@BFriedmanDC ""The other officers yell after Jo...",6,"[Annotator_617, Annotator_618, Annotator_619, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, NO, NO]","[-, REPORTED, -, -, -, -]","[[-], [STEREOTYPING-DOMINANCE], [-], [-], [-],...",TRAIN_EN,NO
203201,203201,en,@v1sig0th @PTSDGuts2 Get off your goddamn high...,6,"[Annotator_508, Annotator_509, Annotator_510, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, YES, NO]","[-, DIRECT, -, -, DIRECT, -]","[[-], [SEXUAL-VIOLENCE], [-], [-], [STEREOTYPI...",TRAIN_EN,NO
203202,203202,en,I hate the phrase “when you assume it makes an...,6,"[Annotator_703, Annotator_704, Annotator_705, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, YES, NO, NO]","[-, DIRECT, -, DIRECT, -, -]","[[-], [OBJECTIFICATION], [-], [STEREOTYPING-DO...",TRAIN_EN,NO
203205,203205,en,To the guy who beeped at me after I'd put my h...,6,"[Annotator_645, Annotator_646, Annotator_647, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, NO, NO, NO, YES, NO]","[REPORTED, -, -, -, DIRECT, -]","[[STEREOTYPING-DOMINANCE], [-], [-], [-], [STE...",TRAIN_EN,NO


In [11]:
frames = [train_Yes, train_No]
result = pd.concat(frames)
result.reset_index(drop=True)

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split,label1
0,200002,en,Writing a uni essay in my local pub with a cof...,6,"[Annotator_391, Annotator_392, Annotator_393, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, NO, YES, YES]","[REPORTED, DIRECT, REPORTED, -, REPORTED, REPO...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION, MIS...",TRAIN_EN,YES
1,200003,en,@UniversalORL it is 2021 not 1921. I dont appr...,6,"[Annotator_397, Annotator_398, Annotator_399, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, NO, YES, NO, YES]","[REPORTED, REPORTED, -, REPORTED, -, JUDGEMENTAL]","[[OBJECTIFICATION, SEXUAL-VIOLENCE], [STEREOTY...",TRAIN_EN,YES
2,200006,en,According to a customer I have plenty of time ...,6,"[Annotator_409, Annotator_410, Annotator_411, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, REPORTED, REPORTED, REPORTED, REPOR...","[[STEREOTYPING-DOMINANCE, OBJECTIFICATION], [S...",TRAIN_EN,YES
3,200007,en,"So only 'blokes' drink beer? Sorry, but if you...",6,"[Annotator_415, Annotator_416, Annotator_417, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[JUDGEMENTAL, REPORTED, REPORTED, DIRECT, DIRE...","[[STEREOTYPING-DOMINANCE], [STEREOTYPING-DOMIN...",TRAIN_EN,YES
4,200011,en,#EverydaySexism means women usually end up in ...,6,"[Annotator_437, Annotator_438, Annotator_439, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, YES, YES, YES, YES]","[REPORTED, JUDGEMENTAL, JUDGEMENTAL, JUDGEMENT...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",TRAIN_EN,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
2865,203180,en,"@BFriedmanDC ""The other officers yell after Jo...",6,"[Annotator_617, Annotator_618, Annotator_619, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, NO, NO]","[-, REPORTED, -, -, -, -]","[[-], [STEREOTYPING-DOMINANCE], [-], [-], [-],...",TRAIN_EN,NO
2866,203201,en,@v1sig0th @PTSDGuts2 Get off your goddamn high...,6,"[Annotator_508, Annotator_509, Annotator_510, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, YES, NO]","[-, DIRECT, -, -, DIRECT, -]","[[-], [SEXUAL-VIOLENCE], [-], [-], [STEREOTYPI...",TRAIN_EN,NO
2867,203202,en,I hate the phrase “when you assume it makes an...,6,"[Annotator_703, Annotator_704, Annotator_705, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, YES, NO, NO]","[-, DIRECT, -, DIRECT, -, -]","[[-], [OBJECTIFICATION], [-], [STEREOTYPING-DO...",TRAIN_EN,NO
2868,203205,en,To the guy who beeped at me after I'd put my h...,6,"[Annotator_645, Annotator_646, Annotator_647, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, NO, NO, NO, YES, NO]","[REPORTED, -, -, -, DIRECT, -]","[[STEREOTYPING-DOMINANCE], [-], [-], [-], [STE...",TRAIN_EN,NO


In [15]:
from gensim.parsing.preprocessing import remove_stopwords

In [16]:
def stopword_removal(row):
    text = row['tweet']
    text = remove_stopwords(text)
    
    return text

In [17]:
import re
result['tweet'] = result.apply(stopword_removal, axis=1)
result['tweet'] = result['tweet'].str.lower()
result.tweet = result.tweet.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
result.tweet.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
result.tweet = result.tweet.apply(lambda x: re.sub(r'&[a-z]+;', '', x))
result.tweet = result.tweet.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
result.tweet = result.tweet.apply(lambda x: re.sub(r'@mention', '', x))
dev['tweet'] = dev.apply(stopword_removal, axis=1)
dev['tweet'] = dev['tweet'].str.lower()
dev.tweet = dev.tweet.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
dev.tweet.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
dev.tweet = dev.tweet.apply(lambda x: re.sub(r'&[a-z]+;', '', x))
dev.tweet = dev.tweet.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
dev.tweet = dev.tweet.apply(lambda x: re.sub(r'@mention', '', x))

In [19]:
vocab_size = 5000 # make the top list of words (common words)
embedding_dim = 50
max_length = 60
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # OOV = Out of Vocabulary

In [49]:
labels = ['YES', 'NO']

In [50]:
train_articles = list(result.tweet)
train_labels = list(result.label1)
validation_articles = list(dev.tweet)
validation_labels =list(dev.label1)
test_labels =[]
test_articles =[]


In [51]:
print(train_articles[0])
print(train_labels[0])

writing uni essay local pub coffee random old man keeps asking drunk questions i'm trying concentrate  ends good luck you'll end getting married use anyway #everydaysexism alive 
YES


In [52]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index

In [53]:
print(word_index)

{'<OOV>': 1, 'i': 2, 'women': 3, 'like': 4, 'the': 5, 'men': 6, 'you': 7, 'woman': 8, 'people': 9, 'it': 10, "don't": 11, 'know': 12, 'man': 13, 'this': 14, "it's": 15, 'its': 16, 'dont': 17, 'look': 18, 'think': 19, 'and': 20, 'gender': 21, 'im': 22, 'want': 23, 'need': 24, 'good': 25, 'time': 26, "i'm": 27, 'way': 28, 'love': 29, 'a': 30, 'sex': 31, 'that': 32, 'day': 33, 'right': 34, 'got': 35, 'sexual': 36, 'fuck': 37, 'if': 38, 'not': 39, 'me': 40, 'feminism': 41, 'so': 42, 'cant': 43, 'work': 44, 'life': 45, 'to': 46, 'my': 47, 'hate': 48, 'ass': 49, 'said': 50, 'but': 51, 'black': 52, 'is': 53, 'in': 54, 'feel': 55, 'new': 56, 'going': 57, 'they': 58, 'he': 59, 'sexist': 60, 'saying': 61, 'what': 62, 'girl': 63, 'girls': 64, 'thing': 65, 'bitch': 66, 'white': 67, 'stop': 68, 'real': 69, 'no': 70, "you're": 71, 'we': 72, 'why': 73, 'cock': 74, 'she': 75, 'shit': 76, 'understand': 77, 'world': 78, 'female': 79, 'do': 80, 'things': 81, 'all': 82, 'tell': 83, 'society': 84, 'identit

In [54]:
train_sequences = tokenizer.texts_to_sequences(train_articles)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)tr
validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
                             

(2870, 60)

In [55]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

In [56]:
print(set(labels))

{'NO', 'YES'}


In [57]:
label_tokenizer.word_index

{'yes': 1, 'no': 2}

In [58]:
print(training_label_seq.shape)
print('-------------')
print(validation_label_seq.shape)

(2870, 1)
-------------
(444, 1)


In [66]:

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
#model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(embedding_dim)))
model.add(Dense(7, activation='softmax'))
model.summary()

2023-05-02 19:36:50.907775: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-02 19:36:50.909456: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-02 19:36:50.910686: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 50)          250000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 100)              40400     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 7)                 707       
                                                                 
Total params: 291,107
Trainable params: 291,107
Non-trainable params: 0
_________________________________________________________________


2023-05-02 19:36:51.119811: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-02 19:36:51.121531: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-02 19:36:51.122797: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [62]:
opt =tf.keras.optimizers.legacy.Adam(learning_rate=0.001, decay=1e-6)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

In [65]:
num_epochs = 20
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)

Epoch 1/20
90/90 - 2s - loss: 5.0468e-05 - accuracy: 1.0000 - val_loss: 2.6411 - val_accuracy: 0.7005 - 2s/epoch - 20ms/step
Epoch 2/20
90/90 - 2s - loss: 4.4954e-05 - accuracy: 1.0000 - val_loss: 2.6723 - val_accuracy: 0.7005 - 2s/epoch - 20ms/step
Epoch 3/20
90/90 - 2s - loss: 4.4519e-05 - accuracy: 1.0000 - val_loss: 2.6777 - val_accuracy: 0.7005 - 2s/epoch - 20ms/step
Epoch 4/20
90/90 - 2s - loss: 3.9675e-05 - accuracy: 1.0000 - val_loss: 2.6860 - val_accuracy: 0.7072 - 2s/epoch - 20ms/step
Epoch 5/20
90/90 - 2s - loss: 3.9714e-05 - accuracy: 1.0000 - val_loss: 2.7049 - val_accuracy: 0.6959 - 2s/epoch - 20ms/step
Epoch 6/20
90/90 - 2s - loss: 3.5431e-05 - accuracy: 1.0000 - val_loss: 2.7335 - val_accuracy: 0.6982 - 2s/epoch - 20ms/step
Epoch 7/20
90/90 - 2s - loss: 3.3546e-05 - accuracy: 1.0000 - val_loss: 2.7375 - val_accuracy: 0.7005 - 2s/epoch - 20ms/step
Epoch 8/20
90/90 - 2s - loss: 3.2210e-05 - accuracy: 1.0000 - val_loss: 2.7649 - val_accuracy: 0.6982 - 2s/epoch - 20ms/step
